# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 30 2022 8:30AM,254027,10,MSP217113,"Methapharm, Inc.",Released
1,Dec 30 2022 8:30AM,254031,10,SOTAH0000872,"Nextsource Biotechnology, LLC",Released
2,Dec 30 2022 8:30AM,254026,10,SNAP33833535,"Snap Medical Industries, LLC",Released
3,Dec 29 2022 4:09PM,254030,19,ADV80009819,"AdvaGen Pharma, Ltd",Released
4,Dec 29 2022 4:09PM,254030,19,ADV80009820,"AdvaGen Pharma, Ltd",Released
5,Dec 29 2022 4:09PM,254030,19,ADV80009821,"AdvaGen Pharma, Ltd",Released
6,Dec 29 2022 4:09PM,254030,19,ADV80009822,"AdvaGen Pharma, Ltd",Released
7,Dec 29 2022 4:09PM,254030,19,ADV80009823,"AdvaGen Pharma, Ltd",Released
8,Dec 29 2022 4:09PM,254030,19,ADV80009824,"AdvaGen Pharma, Ltd",Released
9,Dec 29 2022 4:09PM,254030,19,ADV80009825,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,254019,Released,1
18,254026,Released,1
19,254027,Released,1
20,254030,Released,22
21,254031,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
254019,NaN,1.0
254026,NaN,1.0
254027,NaN,1.0
254030,NaN,22.0
254031,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253953,0.0,1.0
253960,0.0,1.0
253964,0.0,4.0
253982,0.0,3.0
253983,1.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253953,0,1
253960,0,1
253964,0,4
253982,0,3
253983,1,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253953,0,1
1,253960,0,1
2,253964,0,4
3,253982,0,3
4,253983,1,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253953,,1
1,253960,,1
2,253964,,4
3,253982,,3
4,253983,1,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc."
1,Dec 30 2022 8:30AM,254031,10,"Nextsource Biotechnology, LLC"
2,Dec 30 2022 8:30AM,254026,10,"Snap Medical Industries, LLC"
3,Dec 29 2022 4:09PM,254030,19,"AdvaGen Pharma, Ltd"
25,Dec 29 2022 1:47PM,254015,10,ISDIN Corporation
74,Dec 29 2022 1:46PM,254017,10,"Methapharm, Inc."
84,Dec 29 2022 1:46PM,254019,10,Emerginnova
85,Dec 29 2022 1:45PM,254016,10,ISDIN Corporation
88,Dec 29 2022 1:41PM,254012,10,ISDIN Corporation
128,Dec 29 2022 1:41PM,254010,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc.",,1
1,Dec 30 2022 8:30AM,254031,10,"Nextsource Biotechnology, LLC",,1
2,Dec 30 2022 8:30AM,254026,10,"Snap Medical Industries, LLC",,1
3,Dec 29 2022 4:09PM,254030,19,"AdvaGen Pharma, Ltd",,22
4,Dec 29 2022 1:47PM,254015,10,ISDIN Corporation,,49
5,Dec 29 2022 1:46PM,254017,10,"Methapharm, Inc.",,10
6,Dec 29 2022 1:46PM,254019,10,Emerginnova,,1
7,Dec 29 2022 1:45PM,254016,10,ISDIN Corporation,,3
8,Dec 29 2022 1:41PM,254012,10,ISDIN Corporation,,40
9,Dec 29 2022 1:41PM,254010,10,ISDIN Corporation,,32


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc.",1,
1,Dec 30 2022 8:30AM,254031,10,"Nextsource Biotechnology, LLC",1,
2,Dec 30 2022 8:30AM,254026,10,"Snap Medical Industries, LLC",1,
3,Dec 29 2022 4:09PM,254030,19,"AdvaGen Pharma, Ltd",22,
4,Dec 29 2022 1:47PM,254015,10,ISDIN Corporation,49,
5,Dec 29 2022 1:46PM,254017,10,"Methapharm, Inc.",10,
6,Dec 29 2022 1:46PM,254019,10,Emerginnova,1,
7,Dec 29 2022 1:45PM,254016,10,ISDIN Corporation,3,
8,Dec 29 2022 1:41PM,254012,10,ISDIN Corporation,40,
9,Dec 29 2022 1:41PM,254010,10,ISDIN Corporation,32,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc.",1,
1,Dec 30 2022 8:30AM,254031,10,"Nextsource Biotechnology, LLC",1,
2,Dec 30 2022 8:30AM,254026,10,"Snap Medical Industries, LLC",1,
3,Dec 29 2022 4:09PM,254030,19,"AdvaGen Pharma, Ltd",22,
4,Dec 29 2022 1:47PM,254015,10,ISDIN Corporation,49,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc.",1.0,NaN
1,Dec 30 2022 8:30AM,254031,10,"Nextsource Biotechnology, LLC",1.0,NaN
2,Dec 30 2022 8:30AM,254026,10,"Snap Medical Industries, LLC",1.0,NaN
3,Dec 29 2022 4:09PM,254030,19,"AdvaGen Pharma, Ltd",22.0,NaN
4,Dec 29 2022 1:47PM,254015,10,ISDIN Corporation,49.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 30 2022 8:30AM,254027,10,"Methapharm, Inc.",1.0,0.0
1,Dec 30 2022 8:30AM,254031,10,"Nextsource Biotechnology, LLC",1.0,0.0
2,Dec 30 2022 8:30AM,254026,10,"Snap Medical Industries, LLC",1.0,0.0
3,Dec 29 2022 4:09PM,254030,19,"AdvaGen Pharma, Ltd",22.0,0.0
4,Dec 29 2022 1:47PM,254015,10,ISDIN Corporation,49.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4318022,179.0,1.0
19,508018,22.0,1.0
21,508001,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4318022,179.0,1.0
1,19,508018,22.0,1.0
2,21,508001,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,179.0,1.0
1,19,22.0,1.0
2,21,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,179.0
1,19,Released,22.0
2,21,Released,1.0
3,10,Executing,1.0
4,19,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,21
Status,,,
Executing,1.0,1.0,1.0
Released,179.0,22.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,21
0,Executing,1.0,1.0,1.0
1,Released,179.0,22.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,21
0,Executing,1.0,1.0,1.0
1,Released,179.0,22.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()